In [1]:
import copy
import typing as t
from pathlib import Path

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn import metrics
from torch.utils.data import DataLoader, Dataset, Subset, random_split
from torchvision import transforms
from torchvision.datasets import ImageFolder
from tqdm import tqdm

In [2]:
# @formatter:off
%matplotlib inline
# @formatter:on

In [3]:
torch.set_warn_always(True)

sns.set_theme()
plt.rcParams["figure.figsize"] = (8, 4)

In [4]:
DATA_DIR = Path("../data/")

CUDA = "cuda"
CPU = "cpu"
DEVICE = CUDA if torch.cuda.is_available() else CPU
print(f"Using {DEVICE.upper()} device")

Using CUDA device


## Предобработка данных и подготовка датасета

In [5]:
def preprocess_images(dataset: ImageFolder, save_dir: Path) -> None:
    save_dir.mkdir(exist_ok=True)
    for cls in dataset.classes:
        (save_dir / cls).mkdir(exist_ok=True)

    for i in tqdm(range(len(dataset))):
        img, target = dataset[i]
        img.save(save_dir / dataset.classes[target] / Path(dataset.imgs[i][0]).name)


def show_sample_images(dataset: ImageFolder, cols: int = 3, rows: int = 3) -> None:
    _, axes = plt.subplots(cols, rows, figsize=(6, 6))
    for i, ax in zip(torch.randperm(len(dataset)), axes.flatten()):
        img, target = dataset[i]
        ax.axis("off")
        ax.set_title(dataset.classes[target])
        if img.mode == "RGB":
            ax.imshow(img)
        else:
            ax.imshow(img, cmap="gray")


def torch_train_test_split(dataset: t.Union[Dataset, t.Sized], test_size: float) -> t.Tuple[Subset, Subset]:
    test_size = round(test_size * len(dataset))
    train_size = len(dataset) - test_size
    train_dataset, test_dataset = random_split(dataset, lengths=(train_size, test_size))
    return train_dataset, test_dataset

In [ ]:
base_transform = transforms.Compose([
    transforms.Resize(size=(28, 28)),
    transforms.Grayscale(),
])

letters_dataset = ImageFolder(DATA_DIR / "images/eng_handwritten", transform=base_transform)
save_dataset_dir = DATA_DIR / "images/eng_handwritten_48"
preprocess_images(letters_dataset, save_dataset_dir)
letters_dataset = ImageFolder(save_dataset_dir)

 24%|██▍       | 682/2860 [00:15<00:36, 59.81it/s]

In [ ]:
vc = pd.value_counts(letters_dataset.targets)
vc.index = [letters_dataset.classes[i] for i in vc.index]
vc.plot.bar(rot=0, figsize=(5, 3));

In [ ]:
torch.manual_seed(0)
show_sample_images(letters_dataset)

In [ ]:
torch.manual_seed(0)

train_transform = transforms.Compose([
    *base_transform.transforms,
    transforms.RandomHorizontalFlip(p=0.25),
    transforms.RandomVerticalFlip(p=0.25),
    transforms.RandomRotation(degrees=(-30, 30)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,)),
])
test_transform = transforms.Compose([
    *base_transform.transforms,
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5,), std=(0.5,)),
])

train_dataset, test_dataset = torch_train_test_split(letters_dataset, test_size=0.2)
train_dataset.dataset = copy.deepcopy(train_dataset.dataset)
train_dataset.dataset.transform = train_transform
test_dataset.dataset = copy.deepcopy(test_dataset.dataset)
test_dataset.dataset.transform = test_transform

len(train_dataset), len(test_dataset)

## Построение и обучение модели

In [ ]:
def get_weights(targets: torch.Tensor) -> torch.Tensor:
    _, counts = targets.unique(return_counts=True)
    return counts.max() / counts


def common_train(
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        epochs: int,
        train_dataloader: DataLoader,
        test_dataloader: DataLoader,
        verbose: int = None,
        device: str = CPU,
) -> t.Tuple[t.List[float], t.List[float], t.List[float], t.List[float]]:
    train_losses, train_accuracy_list = [], []
    test_losses, test_accuracy_list = [], []
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}\n" + "-" * 32)

        train_loss, train_accuracy = train_loop(train_dataloader, model, loss_fn, optimizer, verbose, device)
        print(f"Train Error: loss: {train_loss:.6f}, accuracy: {train_accuracy:.4f}")
        train_losses.append(train_loss)
        train_accuracy_list.append(train_accuracy)

        test_loss, test_accuracy = test_loop(test_dataloader, model, loss_fn, device)
        print(f" Test Error: loss: {test_loss:.6f}, accuracy: {test_accuracy:.4f}\n")
        test_losses.append(test_loss)
        test_accuracy_list.append(test_accuracy)

        torch.cuda.empty_cache()
    return train_losses, train_accuracy_list, test_losses, test_accuracy_list


def train_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        optimizer: optim.Optimizer,
        verbose: int = None,
        device: str = CPU,
) -> t.Tuple[float, float]:
    model.train()

    size = len(dataloader.dataset)  # noqa
    num_batches = len(dataloader)
    avg_loss, avg_accuracy = 0, 0

    for batch, (x, y) in enumerate(dataloader):
        x, y = x.to(device), y.to(device)

        pred = model(x)
        loss = loss_fn(pred, y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        y_true = torch.flatten(y).detach().cpu()
        y_pred = torch.flatten(pred.argmax(1)).detach().cpu()
        accuracy = metrics.accuracy_score(y_true, y_pred)

        avg_loss += loss
        avg_accuracy += accuracy
        if verbose and batch % verbose == 0:
            print(f"[{batch * len(x):>4d}/{size:>4d}]: loss: {loss:.6f}, accuracy: {accuracy:.4f}")

        del x, y, pred, loss
        torch.cuda.empty_cache()

    return (avg_loss / num_batches).item(), avg_accuracy / num_batches


@torch.no_grad()
def test_loop(
        dataloader: DataLoader,
        model: nn.Module,
        loss_fn: nn.Module,
        device: str = CPU,
) -> t.Tuple[float, float]:
    model.eval()
    y_true, y_pred = get_y_true_y_pred(model, dataloader, device)
    return loss_fn(y_pred, y_true).item(), metrics.accuracy_score(y_true.cpu(), y_pred.argmax(1).cpu())


@torch.no_grad()
def get_y_true_y_pred(
        model: nn.Module,
        dataloader: DataLoader,
        device: str = CPU,
) -> t.Tuple[torch.Tensor, torch.Tensor]:
    model.eval()

    y_test = []
    y_pred = []
    for x, y in dataloader:
        x, y = x.to(device), y.to(device)
        pred = model(x)
        y_test.append(y.unsqueeze(1))
        y_pred.append(pred)

        del x
        torch.cuda.empty_cache()

    return torch.flatten(torch.vstack(y_test)), torch.vstack(y_pred)


def plot_train_test(
        train_losses: t.List[float],
        train_accuracy: t.List[float],
        test_losses: t.List[float],
        test_accuracy: t.List[float],
) -> None:
    fig, axes = plt.subplots(2, 1, figsize=(6, 7))
    epochs = torch.arange(len(train_losses))

    axes[0].plot(epochs, train_losses)
    axes[0].plot(epochs, test_losses)
    axes[0].set_ylabel("loss")
    axes[0].legend(["train", "test"])

    axes[1].plot(epochs, train_accuracy)
    axes[1].plot(epochs, test_accuracy)
    axes[1].set_xlabel("epoch")
    axes[1].set_ylabel("accuracy")

In [ ]:
class CharsClassifier(nn.Module):
    LAST_CONV_OUT_CHANNELS = 64
    ADAPTIVE_AVG_POOL = 4

    def __init__(self, num_channels: int, num_classes: int):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(in_channels=num_channels, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(True),
            nn.MaxPool2d(2),
            nn.Conv2d(in_channels=32, out_channels=self.LAST_CONV_OUT_CHANNELS, kernel_size=3, stride=1, padding=1),
            nn.BatchNorm2d(self.LAST_CONV_OUT_CHANNELS),
            nn.ReLU(True),
            nn.MaxPool2d(2),
        )
        self.avgpool = nn.AdaptiveAvgPool2d((self.ADAPTIVE_AVG_POOL, self.ADAPTIVE_AVG_POOL))
        self.classifier = nn.Linear(
            self.LAST_CONV_OUT_CHANNELS * self.ADAPTIVE_AVG_POOL * self.ADAPTIVE_AVG_POOL,
            num_classes,
        )

    def forward(self, x):
        x = self.features(x)
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        return self.classifier(x)

In [ ]:
torch.manual_seed(0)

net = CharsClassifier(
    num_channels=train_dataset[0][0].size(0),
    num_classes=len(letters_dataset.classes),
).to(DEVICE)
weights = get_weights(torch.tensor([target for _, target in train_dataset], device=DEVICE))
loss_fn = nn.CrossEntropyLoss(weight=weights)
optimizer = optim.Adam(net.parameters(), lr=0.001)

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=512, num_workers=2)

net

In [ ]:
%%time

if __name__ == '__main__':
    train_losses, train_accuracy, test_losses, test_accuracy = common_train(
        epochs=40,
        model=net,
        loss_fn=loss_fn,
        optimizer=optimizer,
        train_dataloader=train_dataloader,
        test_dataloader=test_dataloader,
        device=DEVICE,
    )

## Оценка и выводы

In [ ]:
plot_train_test(train_losses, train_accuracy, test_losses, test_accuracy)  # noqa

In [ ]:
y_true, y_pred = get_y_true_y_pred(net, test_dataloader, DEVICE)
y_true, y_pred = y_true.cpu(), y_pred.argmax(1).cpu()

In [ ]:
cm_display = metrics.ConfusionMatrixDisplay.from_predictions(
    y_true,
    y_pred,
    display_labels=letters_dataset.classes,
    colorbar=False,
    xticks_rotation=0,
    cmap=sns.color_palette('light:b', as_cmap=True)
)
cm_display.ax_.grid(False)
cm_display.figure_.set_size_inches(8, 8)

In [ ]:
print(metrics.classification_report(y_true, y_pred, target_names=letters_dataset.classes, zero_division=True))

In [ ]:
net.eval()

_, axes = plt.subplots(3, 3, figsize=(6, 7))
for i, ax in zip(torch.randperm(len(test_dataset)), axes.flatten()):
    img, _ = letters_dataset[test_dataset.indices[i]]  # картинка до нормализации

    x, target = test_dataset[i]
    pred = net(x.unsqueeze(0).to(DEVICE))

    pred_proba = torch.max(torch.softmax(pred, 1))
    pred_label = pred.argmax(1).item()
    ax.set_title(
        f"pred: {letters_dataset.classes[pred_label]} ({pred_proba:.2f})"
        f"\ntrue: {letters_dataset.classes[target]}", loc="left"
    )
    ax.axis("off")
    if img.mode == "RGB":
        ax.imshow(img)
    else:
        ax.imshow(img, cmap="gray")